# Fitting beta drift to gradient of DLM wind field

This investigates the relationship between broadscale horizontal shear and $\beta$ drift in TC motion. The concept was proposed in Shan and Yu (2020).

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.stats import t
import seaborn as sns
sns.set_context("talk")

# Define the model function
def model(shear, gamma, W):
    #return W + gamma * shear
    return W *(1 + np.exp(gamma * (shear)))

# Function to pass combined Uy and Vx as a single argument to curve_fit
def model_combined(X, gamma, W):
    Uy, Vx = X
    return model(Uy + Vx, gamma, W)

df = pd.read_csv("X:/georisk/HaRIA_B_Wind/projects/tcha/data/derived/envflow/cyclic/beta.components.SH.n20.csv")

In [ ]:
u = df.ub.values
v = df.vb.values
Uy = df.dudy.values*10e5
Vx = df.dvdx.values*10e5
guess = [1, 1]

In [ ]:
uopt, ucov = curve_fit(model_combined, (Uy, Vx), u, p0=guess)
vopt, vcov = curve_fit(model_combined, (Uy, Vx), v, p0=guess)

ugamma_est, uW_est = uopt
vgamma_est, vW_est = vopt

uerr = np.sqrt(np.diag(ucov))
verr = np.sqrt(np.diag(vcov))

alpha = 0.05
n = len(u)
p = len(uopt)
dof = max(0, n-p)
tval = t.ppf(1.0 - alpha / 2, dof)
ugamma_ci = [ugamma_est - tval * uerr[0], ugamma_est + tval * uerr[0]]
vgamma_ci = [vgamma_est - tval * verr[0], vgamma_est + tval * verr[0]]

uW_ci = [uW_est - tval * uerr[1], uW_est + tval * uerr[1]]
vW_ci = [vW_est - tval * verr[1], vW_est + tval * verr[1]]

print(f"Estimated gamma: {ugamma_est}, {vgamma_est}")
print(f"Estimated W: {uW_est}, {vW_est}")

In [ ]:
z = np.sort(Uy + Vx)
u_pred = model(z, ugamma_est, uW_est)
v_pred = model(z, vgamma_est, vW_est)


upper_u = model(z, ugamma_ci[1], uW_ci[1])
lower_u = model(z, ugamma_ci[0], uW_ci[0])

upper_v = model(z, vgamma_ci[1], vW_ci[1])
lower_v = model(z, vgamma_ci[0], vW_ci[0])

# Plot the observed data and the fitted model

fig, ax= plt.subplots(2, 1, sharex=True, figsize=(12, 10))
ax[0].scatter(Uy + Vx, u, label='Observed Data', color='blue')
ax[0].plot(z, u_pred, label='Fitted Model', color='red')
ax[0].fill_between(z, lower_u, upper_u, color='0.5', alpha=0.5)
ax[1].scatter(Uy + Vx, v, label='Observed Data', color='blue')
ax[1].plot(z, v_pred, label='Fitted Model', color='red')
ax[1].fill_between(z, lower_v, upper_v, color='0.5', alpha=0.5)

ax[1].set_xlabel(r'$\partial U/\partial y + \partial V/\partial x \times 10^{-5}$ [s$^{-1}$]')
ax[0].set_ylabel(r'$u_\beta$')
ax[1].set_ylabel(r'$v_\beta$')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
usst = np.sum((u - np.mean(u))**2)
vsst = np.sum((v - np.mean(v))**2)
ussr = np.sum((u - u_pred)**2)
vssr = np.sum((v - v_pred)**2)
ursq = 1 - (ussr / usst)
vrsq = 1 - (vssr / vsst)

print(f"u R-squared: {ursq}")
print(f"v R-squared: {vrsq}")